In [1]:
import glob
import pympi
import numpy as np
import os
import pandas as pd

In [50]:

# ELAN annotation tier names
eaf_tier_names = [
    'mouth_open',

    'food_to_mouth',
    'food_entered',
    'food_lifted',

    'drink_to_mouth',
    'drink_entered',
    'drink_lifted',

    'napkin_to_mouth',
    'napkin_entered',
    'napkin_lifted',

    'disruption'
]


In [3]:
annotations_folder = f'/home/aa2375/social-dining/data/annotation/annotation-files'
annotation_files = sorted(glob.glob(f'{annotations_folder}/*.eaf'))

In [17]:
print(annotation_files)
# For all annotation files

positive_samples = {}
for i, filename in enumerate(annotation_files):
    eaf_obj = pympi.Elan.Eaf(annotation_files[i])
    tier_name = 'food_lifted'
    # tier_name = 'food_to_mouth'
    assert tier_name in eaf_obj.get_tier_names(), f"WARNING: no '{tier_name}' annotations found for: {annotation_files[i]}"

    name = os.path.basename(filename)[:-4]
    
    positive_samples[name] = []
    
    # times are in milliseconds
    for (start_time, end_time, value) in eaf_obj.get_annotation_data_for_tier(tier_name):
        positive_samples[name].append(int(start_time*.030) - 6*30)

['/home/aa2375/social-dining/data/annotation/annotation-files/01_1.eaf', '/home/aa2375/social-dining/data/annotation/annotation-files/01_2.eaf', '/home/aa2375/social-dining/data/annotation/annotation-files/01_3.eaf', '/home/aa2375/social-dining/data/annotation/annotation-files/02_1.eaf', '/home/aa2375/social-dining/data/annotation/annotation-files/02_2.eaf', '/home/aa2375/social-dining/data/annotation/annotation-files/02_3.eaf', '/home/aa2375/social-dining/data/annotation/annotation-files/03_1.eaf', '/home/aa2375/social-dining/data/annotation/annotation-files/03_2.eaf', '/home/aa2375/social-dining/data/annotation/annotation-files/03_3.eaf', '/home/aa2375/social-dining/data/annotation/annotation-files/04_1.eaf', '/home/aa2375/social-dining/data/annotation/annotation-files/04_2.eaf', '/home/aa2375/social-dining/data/annotation/annotation-files/04_3.eaf', '/home/aa2375/social-dining/data/annotation/annotation-files/05_1.eaf', '/home/aa2375/social-dining/data/annotation/annotation-files/05

In [18]:
# Let positive samples be 3 seconds before the liftoff annotation. This is -90 to the values
# DO NOT RUN THIS. This would be how I would ideally train it, but it's not consistent with Tong's
# for k in positive_samples.keys():
#     positive_samples[k] = [number - 90 for number in positive_samples[k]]


In [52]:
total = 0
non_neg = 0
for k,v in positive_samples.items():
    total += len(v)
    non_neg += np.sum(np.array(v) < 0)
print(total)
print(non_neg)
positive_samples['01_1']

6830
19


TypeError: can only concatenate list (not "int") to list

In [20]:
# to mouth intervals contains when the bite has *ended*
# lifted_intervals note when the liftoff has *started*
lifted_intervals = {}
to_mouth_intervals = {}

for i, filename in enumerate(annotation_files):
    name = os.path.basename(filename)[:-4]
    
    lifted_intervals[name] = []
    to_mouth_intervals[name] = []

    eaf_obj = pympi.Elan.Eaf(annotation_files[i])

    tier_name =  'food_lifted'
    assert tier_name in eaf_obj.get_tier_names(), f"WARNING: no '{tier_name}' annotations found for: {annotation_files[i]}"
    # times are in milliseconds
    for (start_time, end_time, value) in eaf_obj.get_annotation_data_for_tier(tier_name):
        lifted_intervals[name].append(int(start_time*.030))
        
    tier_name = 'food_to_mouth'
    assert tier_name in eaf_obj.get_tier_names(), f"WARNING: no '{tier_name}' annotations found for: {annotation_files[i]}"
    # times are in milliseconds
    for (start_time, end_time, value) in eaf_obj.get_annotation_data_for_tier(tier_name):
        to_mouth_intervals[name].append(int(end_time*.030))
        

In [21]:
print(lifted_intervals['01_1'])
print(to_mouth_intervals['01_1'])

[878, 1291, 2251, 3097, 3425, 4782, 5306, 7072, 8012, 9648, 18437, 19039, 21722, 22683, 24348, 25574, 25982, 26223, 28207, 30045, 30561, 31339, 31759, 33976, 35465, 36091, 40924, 44324, 48745, 49559, 51531, 55017, 56237, 58052, 60794, 62741, 67131, 67401]
[1013, 1408, 2358, 3175, 3539, 4857, 5428, 7167, 8580, 9718, 18613, 19392, 21862, 22849, 24407, 25601, 26026, 26264, 28233, 30162, 30630, 31393, 31997, 34026, 35524, 36235, 41074, 44395, 48809, 49624, 51618, 55163, 56317, 58330, 60845, 62807, 67183, 67460]


In [22]:
print(len(lifted_intervals))

90


In [23]:
# index_list = []
# total=0
# for k, lifted_times in lifted_intervals.items():
#     mouth_times = to_mouth_intervals[k]
#     for i in range(len(lifted_times)-1):
#         # print(len(mouth_times), len(lifted_times))
#         interval = lifted_times[i+1] - mouth_times[i]
#         # print(mouth_times[i], lifted_times[i+1])
#         num = int(interval / (30 * 6))
#         # print(num)
#         if num <= 2:
#             continue # skip this for some reason?
#         else:
#             # next bite is 12 seconds later:
#             # non-intention sample: [bite end frame, end frame+6s]
#             num = num -2 # subtract 2 for some reason?
#             total += 1
#             for j in range(num):
#                 # index_list.append(bite_timing_csv.iloc[i, 1] + 6 * 30 *(j+1))
#                 index_list.append(mouth_times[i] + 6*30*(j+1))
# print(len(index_list))
# total

In [24]:
# Tong's way that generates 2486 samples

ll = []
ss = []
negative_samples = {}
for k, lifted_times in lifted_intervals.items():
    negative_samples[k] = []
    s_frame = lifted_times
    for i in range(len(s_frame) - 1):
        num = int((s_frame[i + 1] - s_frame[i])/931)
        for j in range(num):
            negative_samples[k].append(s_frame[i] + int(j*465+465))
            if s_frame[i] <= 180:
                print(s_frame[i], s_frame[i+1])
                print(negative_samples[k][-1])
            # ss.append(k)


79 1164
544


In [25]:
total = 0
non_neg = 0
for k,v in negative_samples.items():
    total += len(v)
    non_neg += np.sum(np.array(v) < 0)
print(total)
print(non_neg)
negative_samples['01_1']

2486
0


[1756,
 3890,
 5771,
 7537,
 8477,
 10113,
 10578,
 11043,
 11508,
 11973,
 12438,
 12903,
 13368,
 13833,
 19504,
 19969,
 22187,
 23148,
 24813,
 26688,
 27153,
 28672,
 32224,
 32689,
 34441,
 36556,
 37021,
 37486,
 37951,
 38416,
 41389,
 41854,
 42319,
 44789,
 45254,
 45719,
 46184,
 50024,
 50489,
 51996,
 52461,
 52926,
 55482,
 56702,
 58517,
 58982,
 61259,
 61724,
 63206,
 63671,
 64136,
 64601]

In [26]:
# my ideal way of generating negative samples. DO NOT USE

# import random

# times_to_sample = 3
# negative_samples = {}

# total = 0
# for k, lifted_times in lifted_intervals.items():
#     mouth_times = to_mouth_intervals[k]
#     negative_samples[k] = []
    
#     for i in range(1,len(lifted_times)):
#         # random number between v[i-1] and v[i]
#         start = mouth_times[i-1] + 30 # 1 second after
#         end = lifted_times[i] - 180 - 90 # 1 second before the liftoff annotation
        
#         # if there is no 6 second gap, continue
#         if end-start < 180:
#             continue
#         # choose a random 6 second window within this range such that rand+180 < end -> rand < end-180
#         # rand_neg_start_idx = random.randint(start, end-180)
#         rand_neg_start_idx = []
        
#         num_samples = times_to_sample
#         num_space = end-start-180
#         if (num_space-times_to_sample) < 0: # sometimes you can't sample times_to_sample, so we correct for it!
#             num_samples = num_space + 1
        
#         rand_neg_start_idx = random.sample(range(start, end-180+1), num_samples)


#         negative_samples[k] += rand_neg_start_idx
#     negative_samples[k].sort()
#     negative_samples[k] = list(negative_samples[k])
#         # print((end-start)/30)


In [27]:
pos_list = []
for k,v in positive_samples.items():
    person_num = k[-1]
    session_num = k[:2]
    for frame in v:
        pos_list.append([k, session_num, person_num, frame])
        # no clue. this is what happeno clue. this is what happe
neg_list = []
for k,v in negative_samples.items():
    person_num = k[-1]
    session_num = k[:2]
    for frame in v:
        neg_list.append([k, session_num, person_num, frame])

In [28]:
pos_df = pd.DataFrame(pos_list)
pos_df.columns = ['Name', 'Session Number', 'Person Number', 'Start Frame']

neg_df = pd.DataFrame(neg_list)
neg_df.columns = ['Name', 'Session Number', 'Person Number', 'Start Frame']



In [29]:
pos_df

,Name,Session Number,Person Number,Start Frame
0,01_1,01,1,698
1,01_1,01,1,1111
2,01_1,01,1,2071
3,01_1,01,1,2917
4,01_1,01,1,3245
...,...,...,...,...
6825,30_3,30,3,49097
6826,30_3,30,3,49949
6827,30_3,30,3,50474
6828,30_3,30,3,50685


In [30]:
neg_df

,Name,Session Number,Person Number,Start Frame
0,01_1,01,1,1756
1,01_1,01,1,3890
2,01_1,01,1,5771
3,01_1,01,1,7537
4,01_1,01,1,8477
...,...,...,...,...
2481,30_3,30,3,39745
2482,30_3,30,3,40210
2483,30_3,30,3,44329
2484,30_3,30,3,47266


In [31]:
pos_df.to_csv('positive_labels.csv')

In [32]:
neg_df.to_csv('negative_labels.csv')

In [33]:
pos_labels = np.ones(len(pos_df), dtype=int)
pos_df['label'] = pos_labels

neg_labels = np.zeros(len(neg_df), dtype=int)
neg_df['label'] = neg_labels

In [34]:
combined_labels = pd.concat([pos_df, neg_df])

In [35]:
combined_labels.to_csv('labels.csv')

In [49]:
pos_df[((pos_df['Start Frame']) < 0)]

,Name,Session Number,Person Number,Start Frame,label
377,03_2,03,2,-29,1
913,06_3,06,3,-158,1
1174,08_2,08,2,-35,1
1252,09_1,09,1,-26,1
1315,09_2,09,2,-153,1
1429,09_3,09,3,-91,1
2150,11_2,11,2,-98,1
2222,11_3,11,3,-166,1
2320,12_2,12,2,-101,1
2498,13_2,13,2,-161,1


In [37]:
neg_df[((neg_df['Start Frame'] ) < 0)]

,Name,Session Number,Person Number,Start Frame,label


In [45]:
len(pos_df)-19

6811